In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotionsdata/sampleSubmission.csv
/kaggle/input/emotionsdata/data_identification.csv
/kaggle/input/emotionsdata/emotion.csv
/kaggle/input/cleanedtweet/cleaned_tweets.pkl


In [2]:
import json
import matplotlib.pyplot as plt
import gc  
import string
import re
from wordcloud import WordCloud
# import contractions
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import spacy
import re
import emoji
import regex # count features
from textblob import TextBlob

In [3]:
# !pip install smart_open==5.1.0
!pip install --upgrade gensim smart_open


  Obtaining dependency information for smart_open from https://files.pythonhosted.org/packages/fc/d9/d97f1db64b09278aba64e8c81b5d322d436132df5741c518f3823824fae0/smart_open-6.4.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: smart_open
    Found existing installation: smart-open 6.3.0
    Uninstalling smart-open-6.3.0:
      Successfully uninstalled smart-open-6.3.0


In [4]:
import pandas as pd

# Load data_identification DataFrame
data_identification = pd.read_csv('/kaggle/input/emotionsdata/data_identification.csv')
import pandas as pd

# Load emotion_data DataFrame
emotion_data = pd.read_csv('/kaggle/input/emotionsdata/emotion.csv')

# Feature Engineering

In [5]:
# Load the Pickle file into a DataFrame
cleaned_tweet_df = pd.read_pickle('/kaggle/input/cleanedtweet/cleaned_tweets.pkl')

In [6]:
import pandas as pd

cleaned_tweet_df = pd.read_pickle('/kaggle/input/cleanedtweet/cleaned_tweets.pkl')

# Ensure that cleaned_tweet_df is a DataFrame
cleaned_tweet_df = pd.DataFrame(cleaned_tweet_df)

# Print the first few rows
print(cleaned_tweet_df.head())

   tweet_id                                               text  \
0  0x376b20  People who post "add me on #Snapchat" must be ...   
1  0x2d5350  @brianklaas As we see, Trump is dangerous to #...   
2  0x28b412  Confident of your obedience, I write to you, k...   
3  0x1cd5b0                Now ISSA is stalking Tasha 😂😂😂 <LH>   
4  0x2de201  "Trust is not the same as faith. A friend is s...   

                            cleaned_text_with_emojis  
0    people post add I snapchat dehydrate cuz man lh  
1  trump dangerous freepress world lh lh trumpleg...  
2  confident obedience I write know I ask philemo...  
3                                issa stalk tasha lh  
4  trust faith friend trust faith mistake christo...  


In [7]:
# Merge cleaned_tweet_df with emotion_data on 'tweet_id'
cleaned_and_emotion_df = pd.merge(cleaned_tweet_df, emotion_data, on='tweet_id', how='left')

# Display the first few rows of the merged DataFrame
print(cleaned_and_emotion_df.head())

   tweet_id                                               text  \
0  0x376b20  People who post "add me on #Snapchat" must be ...   
1  0x2d5350  @brianklaas As we see, Trump is dangerous to #...   
2  0x28b412  Confident of your obedience, I write to you, k...   
3  0x1cd5b0                Now ISSA is stalking Tasha 😂😂😂 <LH>   
4  0x2de201  "Trust is not the same as faith. A friend is s...   

                            cleaned_text_with_emojis       emotion  
0    people post add I snapchat dehydrate cuz man lh  anticipation  
1  trump dangerous freepress world lh lh trumpleg...       sadness  
2  confident obedience I write know I ask philemo...           NaN  
3                                issa stalk tasha lh          fear  
4  trust faith friend trust faith mistake christo...           NaN  


In [8]:
# Merge the merged_cleaned_and_emotion with data_identification
merged_df = pd.merge(cleaned_and_emotion_df, data_identification, on='tweet_id', how='left')

# Display the first few rows of the final merged dataframe
print(merged_df.head())

   tweet_id                                               text  \
0  0x376b20  People who post "add me on #Snapchat" must be ...   
1  0x2d5350  @brianklaas As we see, Trump is dangerous to #...   
2  0x28b412  Confident of your obedience, I write to you, k...   
3  0x1cd5b0                Now ISSA is stalking Tasha 😂😂😂 <LH>   
4  0x2de201  "Trust is not the same as faith. A friend is s...   

                            cleaned_text_with_emojis       emotion  \
0    people post add I snapchat dehydrate cuz man lh  anticipation   
1  trump dangerous freepress world lh lh trumpleg...       sadness   
2  confident obedience I write know I ask philemo...           NaN   
3                                issa stalk tasha lh          fear   
4  trust faith friend trust faith mistake christo...           NaN   

  identification  
0          train  
1          train  
2           test  
3          train  
4           test  


In [9]:
# Separate the data into training and testing sets
train_df = merged_df[merged_df['identification'] == 'train']
test_df = merged_df[merged_df['identification'] == 'test']

# Display the shapes of the training and testing sets
print("Training Data Shape:", train_df.shape)
print("Testing Data Shape:", test_df.shape)

Training Data Shape: (1455563, 5)
Testing Data Shape: (411972, 5)


In [10]:
test_df.head()

,tweet_id,text,cleaned_text_with_emojis,emotion,identification
2,0x28b412,"Confident of your obedience, I write to you, k...",confident obedience I write know I ask philemo...,NaN,test
4,0x2de201,"""Trust is not the same as faith. A friend is s...",trust faith friend trust faith mistake christo...,NaN,test
9,0x218443,When do you have enough ? When are you satisfi...,satisfied goal money materialism money possess...,NaN,test
30,0x2939d5,"God woke you up, now chase the day #GodsPlan #...",god wake chase day godsplan godswork lh,NaN,test
33,0x26289a,"In these tough times, who do YOU turn to as yo...",tough time turn symbol hope lh,NaN,test


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Drop rows with missing values in the 'emotion' column
train_df_cleaned = train_df.dropna(subset=['emotion'])

# Feature Extraction
tfidf_vectorizer = TfidfVectorizer(max_features=50000, stop_words='english', ngram_range=(1, 3), max_df=0.85, min_df=2, sublinear_tf=True)
X_train = tfidf_vectorizer.fit_transform(train_df_cleaned['cleaned_text_with_emojis'])
y_train = train_df_cleaned['emotion']

# Hyperparameter Tuning with Class Weights
param_grid = {'alpha': [0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 500.0, 1000.0]}  # Adjusted alpha values
grid_search = GridSearchCV(MultinomialNB(), param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train)
best_alpha = grid_search.best_params_['alpha']

# Building and Training the Naive Bayes Model with the best alpha and default class weights
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)
nb_classifier = MultinomialNB(alpha=best_alpha)
nb_classifier.fit(X_train_split, y_train_split)

# Predict on the validation set
y_val_pred = nb_classifier.predict(X_val_split)

# Evaluate the performance on the validation set
print("Validation Accuracy:", accuracy_score(y_val_split, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val_split, y_val_pred))

# Feature extraction for test data
X_test = tfidf_vectorizer.transform(test_df['cleaned_text_with_emojis'])

# Predicting emotions in test data
test_df['predicted_emotion'] = nb_classifier.predict(X_test)

# Creating a submission file
submission_df = test_df[['tweet_id', 'predicted_emotion']].rename(columns={'tweet_id': 'id'})
submission_df.to_csv('submission.csv', index=False)

Validation Accuracy: 0.5210863135620876

Classification Report:
               precision    recall  f1-score   support

       anger       0.61      0.16      0.26      7973
anticipation       0.60      0.51      0.55     49787
     disgust       0.41      0.41      0.41     27820
        fear       0.60      0.34      0.43     12800
         joy       0.53      0.77      0.63    103204
     sadness       0.44      0.43      0.44     38687
    surprise       0.62      0.15      0.25      9746
       trust       0.53      0.27      0.36     41096

    accuracy                           0.52    291113
   macro avg       0.54      0.38      0.42    291113
weighted avg       0.53      0.52      0.50    291113



/tmp/ipykernel_42/1319764661.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['predicted_emotion'] = nb_classifier.predict(X_test)
